Load Excel file

In [93]:
import pandas as pd

In [94]:
PATH="C:\\Users\\nicol\\OneDrive\\Documentos\\GitHub\\timeTablingFisiUNMSM\\Contexto\\"

In [95]:
xls = pd.ExcelFile(PATH+"FORMATO MODELO DISPONIBILIDAD 2.xlsx")
df = xls.parse("Table 1", skiprows=8)

In [96]:
df

,HORA,Lun,Mar,Mie,Jue,Vie,Sab,Dom
0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,14,X,X,X,X,X,NaN,NaN
7,15,X,X,X,X,X,NaN,NaN
8,16,X,X,X,X,X,NaN,NaN
9,17,X,X,X,X,X,NaN,NaN


Set expected column names

In [97]:
df.columns = ['HOUR', 'Lun', 'Mar', 'Mie', 'Jue', 'Vie', 'Sab', 'Dom']

Convert HOUR safely to integers

In [98]:
df['HOUR'] = pd.to_numeric(df['HOUR'], errors='coerce')
df = df[df['HOUR'].notna()]
df['HOUR'] = df['HOUR'].astype(int)


C:\Users\nicol\AppData\Local\Temp\ipykernel_6872\2544343130.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HOUR'] = df['HOUR'].astype(int)


Keep only hours from 8 to 21

In [99]:
df = df[df['HOUR'].between(8, 22)]


Days to extract

In [100]:
days = ['Lun', 'Mar', 'Mie', 'Jue', 'Vie', 'Sab', 'Dom']

Convert hour number to readable format

In [101]:
def to_ampm(h):
    return f"{(h-1)%12+1}{'am' if h < 12 else 'pm'}"

Group continuous ranges

In [102]:
def group_ranges(hours):
    hours = sorted(set(hours))
    ranges = []
    start = prev = hours[0]

    for h in hours[1:]:
        if h == prev + 1:
            prev = h
        else:
            ranges.append((start, prev))
            start = prev = h
    ranges.append((start, prev))
    return [f"{to_ampm(s)} to {to_ampm(e+1)}" if s != e else f"{to_ampm(s)} to {to_ampm(s+1)}" for s, e in ranges]


Extract and group availability

In [103]:
grouped_availability = []

for day in days:
    available_hours = [
        row['HOUR'] for _, row in df.iterrows()
        if str(row[day]).strip().upper() == 'X'
    ]
    if available_hours:
        time_ranges = group_ranges(available_hours)
        for r in time_ranges:
            grouped_availability.append({
                "day": day,
                "time_range": r
            })


Output to JSON

In [104]:
# Convertir la lista de diccionarios a DataFrame
df_availability = pd.DataFrame(grouped_availability)

# Guardar como CSV
df_availability.to_csv("grouped_availability2.csv",sep=";", index=False, encoding="utf-8")

print("✅ Grouped availability saved to grouped_availability.csv")

✅ Grouped availability saved to grouped_availability.csv
